# ginzaの使い方

## 単語の共起

単語が出現する頻度を単独で調べるだけでなく，単語間の関係を調べることもできる．
共起を求める関数を定義し，題材の小説「影男」で共起を求めてみる．

In [1]:
import spacy

input_fn = 'text/kageotoko.corpus.txt'

include_pos = ('NOUN', 'VERB', 'ADJ')
stopwords = ('する', 'ある', 'ない', 'いう', 'もの', 'こと', 'よう', 'なる', 'ほう', 'いる', 'くる')

nlp = spacy.load("ja_ginza")

In [2]:
def extract_words(sent, pos_tags, stopwords):
    words = [token.lemma_ for token in sent
             if token.pos_ in pos_tags and token.lemma_ not in stopwords]
    return words

In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def count_cooccurrence(sents, token_length='{2,}'):
    token_pattern=f'\\b\\w{token_length}\\b'
    count_model = CountVectorizer(token_pattern=token_pattern)

    X = count_model.fit_transform(sents)
    words = count_model.get_feature_names_out()
    word_counts = np.asarray(X.sum(axis=0)).reshape(-1)

    X[X > 0] = 1 # limit to 1 occurrence in a document.
    Xc = (X.T * X) # this is co-occurrence matrix in sparse csr format
    return words, word_counts, Xc, X

In [10]:
def chunk(lst, n):
    """
    リストをn個ずつのサブリストに分割する。
    """
    return [lst[i:i+n] for i in range(0, len(lst), n)]


with open(input_fn, 'r') as f:
    lines = [l.replace("\n","") for l in f.readlines()]

lst_texts = chunk(lines,100)
sents = []
list_sents = []
for text in lst_texts:
    doc = nlp(" ".join(text))
    sents.extend([' '.join(extract_words(sent, include_pos, stopwords))
                  for sent in doc.sents])
    list_sents.extend(doc.sents)

words, _, Xc, X = count_cooccurrence(sents)

共起頻度の高い順に10個表示する．

In [11]:
from collections import Counter
counter = Counter()

for i, j in zip(*Xc.nonzero()):
    if i >= j:
        continue
    counter[(i,j)] += Xc[i,j]

for (i,j), c in counter.most_common(10):
    print('{:>3d} ({}, {})'.format(c, words[i], words[j]))

 22 (会社, 殺人)
 18 (やる, 来る)
 17 (さま, だんな)
 16 (さん, じい)
 16 (ひらく, ドア)
 16 (世界, 地底)
 13 (かける, 電話)
 12 (いく, まま)
 11 (いく, 出る)
 11 (くちびる, 赤い)


「世界」と「地底」が共起する原文を表示する．

In [12]:
def find_sentence_by_cooccurrence(X, idxs):
    occur_flags = (X[:,idxs[0]] > 0)
    for idx in idxs[1:]:
        occur_flags = occur_flags.multiply(X[:,idx] > 0)
    return occur_flags.nonzero()[0]

sents_orig = list_sents
words_lookup = {word: index for index, word in enumerate(words)}
idxs = [words_lookup[word] for word in ['世界', '地底']]

for i in find_sentence_by_cooccurrence(X, idxs):
    print("{:>5d}: {}".format(i, sents_orig[i]))

 2590: そういうおかたは、この地底世界へおつれすることさえむずかしい。
 2660: かれはそこでは、いつものゆすりを行なう気にもならず、地底の主人公のちょびひげ紳士と親交を約して別れをつげ、地上世界に立ち帰った。
 4690: むろん、地底世界のつづきなのだ。
 4718: この地底世界に、それほどの巨資があるのであろうか。
 4719: このまえにちょびひげがいっていたのでは、地底世界の女の数は百人ぐらいのはずであった。
 4759: 　地上世界の見せ物でこんなことをやれば、すぐに種がわかってしまうが、地底の洞窟という好条件がある。
 4789: 　洞窟にはいってから二時間あまり、黒いメフィストは時を忘れ、追われている身を忘れ、地上のいっさいの煩いを忘れ、艶樹の森と、地底世界をどよもす音楽と、歌声と、踊り狂う五面十脚の美しい怪獣とに、果てしもなく酔いしれていたが、ふと気がつくと、またしても、ただならぬ奇怪事が起こっていた。
 4836: どうしてこの地底世界へ、警官がはいりこんできたのか。
 4845: 地底世界の経営者が内通したのだろうか。
 4955: すると、こういうおもしろい地底の世界を見せてくれた。
 4964: 「それにしても、明智先生は、この地底の世界へははじめて来られたのでしょう。
 4976: 　一方、ぼくは地底世界で、ちょっと荒療治をやった。
 4980: 地底世界の様子が、あらましわかった。
 4985: 　それから、ちょびひげを脅迫して、池のシリンダーを浮き上がらせ、待機していた十人の警官を地底世界に引き入れた。
 5044:  この世の果て  　明智小五郎は、中村警部やその部下とともに、地底世界の入り口に近いいわゆる事務室にもどっていた。
 5064: 二つの世界で、わたしの地底王国はいっぱいですよ」
